In [ ]:
"""
Use Case Description : Monitor RUs and performance benefits by partitioning and indexing document properties

1. Create a cosmos client
2. create a new database

"""

In [2]:
from azure.cosmos import CosmosClient, exceptions, PartitionKey


from azure_cli import getcosmosaccountaccesskey, getcosmosaccounturl

In [3]:
#Get the URL and key for the cosmos account

cosmosurl = getcosmosaccounturl(
    cosmosaccountname='raj-cosmos-2020',
    resourcegroup='azurelearning',
    subscription='1e6e9ea7-c0f7-4d7c-bae3-1199a177e943'
    )

accesskey = getcosmosaccountaccesskey(
    cosmosaccountname='raj-cosmos-2020',
    resourcegroup='azurelearning',
    subscription='1e6e9ea7-c0f7-4d7c-bae3-1199a177e943'
    )

In [4]:
#Create a cosmos client

cosmos_client = CosmosClient(url=cosmosurl, credential=accesskey)

In [32]:
# Create a new database for this POC by providing initial RUs as 500

database_client = cosmos_client.create_database_if_not_exists(id="performance_poc",offer_throughput=500)

In [13]:
# verify if database has been created

for db in cosmos_client.list_databases():
    print(db['id'])

Products
performance_poc
demodb
cosmos-poc


In [33]:
# see the response for the request to create a new database

database_client.client_connection.last_response_headers['x-ms-request-charge']

'2'

In [26]:
# sample json data to be used for this POC

poc_data = [
	{
		"id": "0001",
		"type": "donut",
		"name": "Cake",
		"ppu": 0.55,
		"batters":
			{
				"batter":
					[
						{ "id": "1001", "type": "Regular" },
						{ "id": "1002", "type": "Chocolate" },
						{ "id": "1003", "type": "Blueberry" },
						{ "id": "1004", "type": "Devil's Food" }
					]
			},
		"topping":
			[
				{ "id": "5001", "type": "None" },
				{ "id": "5002", "type": "Glazed" },
				{ "id": "5005", "type": "Sugar" },
				{ "id": "5007", "type": "Powdered Sugar" },
				{ "id": "5006", "type": "Chocolate with Sprinkles" },
				{ "id": "5003", "type": "Chocolate" },
				{ "id": "5004", "type": "Maple" }
			]
	},
	{
		"id": "0002",
		"type": "donut",
		"name": "Raised",
		"ppu": 0.55,
		"batters":
			{
				"batter":
					[
						{ "id": "1001", "type": "Regular" }
					]
			},
		"topping":
			[
				{ "id": "5001", "type": "None" },
				{ "id": "5002", "type": "Glazed" },
				{ "id": "5005", "type": "Sugar" },
				{ "id": "5003", "type": "Chocolate" },
				{ "id": "5004", "type": "Maple" }
			]
	},
	{
		"id": "0003",
		"type": "donut",
		"name": "Old Fashioned",
		"ppu": 0.55,
		"batters":
			{
				"batter":
					[
						{ "id": "1001", "type": "Regular" },
						{ "id": "1002", "type": "Chocolate" }
					]
			},
		"topping":
			[
				{ "id": "5001", "type": "None" },
				{ "id": "5002", "type": "Glazed" },
				{ "id": "5003", "type": "Chocolate" },
				{ "id": "5004", "type": "Maple" }
			]
	}
]

In [ ]:
"""
Lets analyse the data above before creating the container as we need to figure out the right partition key(s)

"""

In [21]:
# lets start with using the "id" column as the partition key
partitonkey = PartitionKey(path="/id", kind='hash')

In [22]:
# now lets create a container named "bakery"
container_client = database_client.create_container_if_not_exists(id="bakery", partition_key=partitonkey)

In [23]:
# verify if container has been created

for container in database_client.list_containers():
    print(container['id'])

bakery


In [28]:
# now lets populate the container with documents from our list "poc_data" having json documents
# for now , i have not brought indexes in the picture. i'll do it in the next steps

for document in poc_data:
    #print(document)
    container_client.create_item(body=document)
print("Documents have been populated in collection bakery")

Documents have been populated in collection bakery


In [30]:
# lets see the number of RUs used for inserting these documents in the collection

RUs = container_client.client_connection.last_response_headers['x-ms-request-charge']
print("Number of RUs used for populating documents is {}".format(RUs))

Number of RUs used for populating documents is 12.38


In [40]:
# Lets try quering the collection

"""Here i m trying to retrieve the document against a particular id. since the partition key is /id, 
so ideally this request must be going to a single partition"""

import json

sqlquery = "select * from bakery b where b.id='0001'"
for item in container_client.query_items(query=sqlquery, max_item_count=2):
    print(json.dumps(item, indent=True))


{
 "id": "0001",
 "type": "donut",
 "name": "Cake",
 "ppu": 0.55,
 "batters": {
  "batter": [
   {
    "id": "1001",
    "type": "Regular"
   },
   {
    "id": "1002",
    "type": "Chocolate"
   },
   {
    "id": "1003",
    "type": "Blueberry"
   },
   {
    "id": "1004",
    "type": "Devil's Food"
   }
  ]
 },
 "topping": [
  {
   "id": "5001",
   "type": "None"
  },
  {
   "id": "5002",
   "type": "Glazed"
  },
  {
   "id": "5005",
   "type": "Sugar"
  },
  {
   "id": "5007",
   "type": "Powdered Sugar"
  },
  {
   "id": "5006",
   "type": "Chocolate with Sprinkles"
  },
  {
   "id": "5003",
   "type": "Chocolate"
  },
  {
   "id": "5004",
   "type": "Maple"
  }
 ],
 "_rid": "YMQ3AI3g9zwBAAAAAAAAAA==",
 "_self": "dbs/YMQ3AA==/colls/YMQ3AI3g9zw=/docs/YMQ3AI3g9zwBAAAAAAAAAA==/",
 "_etag": "\"d700583c-0000-0100-0000-5fe2399c0000\"",
 "_attachments": "attachments/",
 "_ts": 1608661404
}


In [41]:
# Find the request units for retreiving one document as per the above step

container_client.client_connection.last_response_headers['x-ms-request-charge']

'2.83'

In [42]:
# Now lets try to query multiple documents 

"""
Here i am trying to query items against two ids 0001 and 0002. since the collection is partitioned using id property, 
this query is going to get the results from multiple partitions
"""

sqlquery = "select * from bakery b where b.id in ('0001','0002')"
for item in container_client.query_items(query=sqlquery, max_item_count=2):
    print(json.dumps(item, indent=True))

CosmosHttpResponseError: (BadRequest) Cross partition query is required but disabled. Please set x-ms-documentdb-query-enablecrosspartition to true, specify x-ms-documentdb-partitionkey, or revise your query to avoid this exception.
ActivityId: c5afcaab-2af8-4802-86d2-3d80c93aa958, Microsoft.Azure.Documents.Common/2.11.0

In [43]:
"""Above error says that cross partition is disabled as it is trying to get the items from multiple partitions
lets resolve that by using a property while querying the items
"""

sqlquery = "select * from bakery b where b.id in ('0001','0002')"
for item in container_client.query_items(query=sqlquery, max_item_count=2, enable_cross_partition_query=True):
    print(item)


{'id': '0001', 'type': 'donut', 'name': 'Cake', 'ppu': 0.55, 'batters': {'batter': [{'id': '1001', 'type': 'Regular'}, {'id': '1002', 'type': 'Chocolate'}, {'id': '1003', 'type': 'Blueberry'}, {'id': '1004', 'type': "Devil's Food"}]}, 'topping': [{'id': '5001', 'type': 'None'}, {'id': '5002', 'type': 'Glazed'}, {'id': '5005', 'type': 'Sugar'}, {'id': '5007', 'type': 'Powdered Sugar'}, {'id': '5006', 'type': 'Chocolate with Sprinkles'}, {'id': '5003', 'type': 'Chocolate'}, {'id': '5004', 'type': 'Maple'}], '_rid': 'YMQ3AI3g9zwBAAAAAAAAAA==', '_self': 'dbs/YMQ3AA==/colls/YMQ3AI3g9zw=/docs/YMQ3AI3g9zwBAAAAAAAAAA==/', '_etag': '"d700583c-0000-0100-0000-5fe2399c0000"', '_attachments': 'attachments/', '_ts': 1608661404}
{'id': '0002', 'type': 'donut', 'name': 'Raised', 'ppu': 0.55, 'batters': {'batter': [{'id': '1001', 'type': 'Regular'}]}, 'topping': [{'id': '5001', 'type': 'None'}, {'id': '5002', 'type': 'Glazed'}, {'id': '5005', 'type': 'Sugar'}, {'id': '5003', 'type': 'Chocolate'}, {'id'

In [44]:
# Find the request units for retreiving one document as per the above step

container_client.client_connection.last_response_headers['x-ms-request-charge']

'2.97'

In [45]:
# lets find the RUs of getting all the data

sqlquery = "select * from bakery b"
for item in container_client.query_items(query=sqlquery, max_item_count=2, enable_cross_partition_query=True):
    pass

container_client.client_connection.last_response_headers['x-ms-request-charge']

'2.27'

In [ ]:
"""
Lets create another collection by using different partition combinations
"""

In [74]:
# define the partition key

partitonkey = PartitionKey(path='/batters/batter', kind='Hash')

# create a new collection
#database_client.delete_container('bakery_2')

container_client_2 = database_client.create_container_if_not_exists(id='bakery_2', partition_key=partitonkey)

In [75]:
# populate the collection

for document in poc_data:
    #print(document)
    container_client_2.create_item(body=document)
print("Documents have been populated in collection bakery2")

Documents have been populated in collection bakery2


In [119]:
# lets find the RUs of getting all the data

sqlquery = "select * from bakery2 b where b.id in ('0001','0002')"
for item in container_client_2.query_items(query=sqlquery, max_item_count=2, enable_cross_partition_query=True):
    print(item)

print(json.dumps(container_client_2.client_connection.last_response_headers['x-ms-request-charge'], indent=True))

{'id': '0001', 'type': 'donut', 'name': 'Cake', 'ppu': 0.55, 'batters': {'batter': [{'id': '1001', 'type': 'Regular'}, {'id': '1002', 'type': 'Chocolate'}, {'id': '1003', 'type': 'Blueberry'}, {'id': '1004', 'type': "Devil's Food"}]}, 'topping': [{'id': '5001', 'type': 'None'}, {'id': '5002', 'type': 'Glazed'}, {'id': '5005', 'type': 'Sugar'}, {'id': '5007', 'type': 'Powdered Sugar'}, {'id': '5006', 'type': 'Chocolate with Sprinkles'}, {'id': '5003', 'type': 'Chocolate'}, {'id': '5004', 'type': 'Maple'}], '_rid': 'YMQ3AJkTu7oBAAAAAAAAAA==', '_self': 'dbs/YMQ3AA==/colls/YMQ3AJkTu7o=/docs/YMQ3AJkTu7oBAAAAAAAAAA==/', '_etag': '"da00f28c-0000-0100-0000-5fe24b4d0000"', '_attachments': 'attachments/', '_ts': 1608665933}
{'id': '0002', 'type': 'donut', 'name': 'Raised', 'ppu': 0.55, 'batters': {'batter': [{'id': '1001', 'type': 'Regular'}]}, 'topping': [{'id': '5001', 'type': 'None'}, {'id': '5002', 'type': 'Glazed'}, {'id': '5005', 'type': 'Sugar'}, {'id': '5003', 'type': 'Chocolate'}, {'id'

In [82]:
container_properties = container_client_2.read()

In [86]:
print("Indexing policy for the container : \n {}".format(json.dumps(container_properties['indexingPolicy'], indent=True)))

Indexing policy for the container : 
 {
 "indexingMode": "consistent",
 "automatic": true,
 "includedPaths": [
  {
   "path": "/*"
  }
 ],
 "excludedPaths": [
  {
   "path": "/\"_etag\"/?"
  }
 ]
}


In [116]:
"""
as we see in the above output, that every (/*) property in the document is indexed.
lets exclude some nested structures from the indexing so that we can save some storage and cost
"""

Indexing_Policy_for_Bakery = {
    
    "indexingMode" : "consistent",
    "automatic" : True,
    "includedPaths" : [{"path" : "/*"}],
    "excludedPaths" : [{"path" : "/\"batters\"/*"}]
}

In [117]:
# lets create another container with this indexing policy and try to query the nodes which are not indexed anymore

database_client.delete_container('bakery_3')

partitonkey = PartitionKey(path='/id', kind='Hash')
container_client_3 = database_client.create_container_if_not_exists(
    id='bakery_3', 
    partition_key=partitonkey,
    indexing_policy=Indexing_Policy_for_Bakery
)

In [118]:
container_properties = container_client_3.read()
print("Indexing policy for the container : \n {}".format(json.dumps(container_properties['indexingPolicy'], indent=True)))

Indexing policy for the container : 
 {
 "indexingMode": "consistent",
 "automatic": true,
 "includedPaths": [
  {
   "path": "/*"
  }
 ],
 "excludedPaths": [
  {
   "path": "/\"batters\"/*"
  },
  {
   "path": "/\"_etag\"/?"
  }
 ]
}
